<a href="https://colab.research.google.com/github/zhangkwn617/M.S.B.A.-Projects/blob/main/Beer%20Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Analysis Homework 2: Recommending Beer
###Justin Wagers
### Daniel Stern
### Kaiwen Zhang
### Tairan Deng
### Maryam Blooki
### Sunny Vidhani


## Task A

Here we scrape 25 reviews for 250 beers for a total of 6,250 reviews from Beer Advocate's top rated beers list. We also scrape the numerical rating for each review. 

In [ ]:
!pip install selenium
!apt-get -q update 
!apt install -yq chromium-chromedriver

     |████████████████████████████████| 911kB 4.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubun

In [ ]:
import sys
import pandas as pd
sys.path.insert(0, '/user/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
driver.get("https://www.beeradvocate.com/beer/top-rated/")
table_rows = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr/td/a')
links = [row.get_attribute('href') for row in table_rows]

reviewsdf = pd.DataFrame(columns = ['product_name','review','user_rating'])
for link in links:
  driver.get(link)
  title = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[3]/div/div/div[1]/h1').text.replace('\n',' - ')
  print(title)
  #ratings_elems = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[1]/div[2]/span[2]')
  ratings_elems = driver.find_elements_by_xpath('//span[@class="BAscore_norm"]')
  ratings = []
  for rating in ratings_elems:
    ratings.append(float(rating.text))
  #reviews_elems = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div/div[2]')
  reviews_elems = driver.find_elements_by_xpath('//*[@id="rating_fullview_content_2"]')
  reviews = []
  for review in reviews_elems:
    lines_list = review.text.split('\n')
    lines_list = lines_list[5:]
    reviews.append((' '.join(lines_list)))
  for review, rating in list(zip(reviews, ratings)):
    reviewsdf.loc[len(reviewsdf)] = [title, review, rating]

Kentucky Brunch Brand Stout - Toppling Goliath Brewing Company
Marshmallow Handjee - 3 Floyds Brewing Co.
Barrel-Aged Abraxas - Perennial Artisan Ales
Hunahpu's Imperial Stout - Double Barrel Aged - Cigar City Brewing
Heady Topper - The Alchemist
Pliny The Younger - Russian River Brewing Company
King Julius - Tree House Brewing Company
Mornin' Delight - Toppling Goliath Brewing Company
Bourbon County Brand Coffee Stout - Goose Island Beer Co.
King JJJuliusss - Tree House Brewing Company
Very Hazy - Tree House Brewing Company
SR-71 - Toppling Goliath Brewing Company
Fundamental Observation - Bottle Logic Brewing


In [ ]:
reviewsdf.to_csv('beer_data.csv', index = False)

In [ ]:
import pandas as pd

beer_data = pd.read_csv(r"training_data (1).csv")
display(beer_data)

## Task B

Here we assume that a specific beer customer has specified three attributes that they are interested in: hoppy, fruity, and sweet. To collect all instances of hoppy and fruity, we abbreviate the attributes to 'hop' and 'fruit'. 

We decided on these attributes after conducting a word frequency analysis on the reviews, shown below. Each of the three attributes occured at least 800 times in the collective set of reviews. 

In [ ]:

import nltk
nltk.download("popular")
from nltk.tokenize import word_tokenize
import string

top_N = 400

a = beer_data['review'].str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)

rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

rslt['Word'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
rslt['Word'].str.replace('[{}]'.format(string.punctuation), '')
display(rslt[0:50])


## Task C

Next we perform cosine similarity with the attributes specified by 'the customer': hop, fruit, and sweet. This gives us a basic measure of how similar each review is to the desired attributes. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def calculate_similarity(attributes_doc, corpus_list, product_list):
  corpus_list.append(attributes_doc)

  # Create the Document Term Matrix
  tfidf_vectorizer = TfidfVectorizer(stop_words='english')
  #tfidf_vectorizer = TfidfVectorizer()
  sparse_matrix = tfidf_vectorizer.fit_transform(corpus_list) # no error up to this point

  doc_term_matrix = sparse_matrix.todense()
  df = pd.DataFrame(doc_term_matrix, 
                    columns=tfidf_vectorizer.get_feature_names()) # no error up to this point

  for i in list(df.columns):
    if df.loc[len(corpus_list)-1,i] == 0:
      df = df.drop(i, axis=1) 

  cosine_result = cosine_similarity(df, df)
  cosine_values_list = []
  for i in range(len(cosine_result)):
    b = cosine_result[i][-1]
    cosine_values_list.append(b)
  
  cosine_values_list = cosine_values_list[:-1]
  similarity_series = pd.Series(cosine_values_list)

  corpus_list.remove(attributes_doc)
  review_series = pd.Series(corpus_list)
  product_series = pd.Series(product_list)

  result_df = pd.concat([product_series, review_series, similarity_series],axis=1)

  result_df.columns = ['product_name', 'review', 'similarity']

  return result_df

In [ ]:
sim = calculate_similarity('hop fruit sweet', list(beer_data['review'].values), list(beer_data['product_name'].values))

In [ ]:
beer_data['similarity'] = sim.similarity

In [ ]:
beer_data.to_csv('beer_data_sim.csv', index = False)

## Task D
Now, we conduct feature level sentiment analysis for each of the three features. Our regex expression below pulls 3 words before and after each instance of an attribute being mentioned, and sentiment analysis is then conducted to determine if these mentions are positive or negative. 

In [ ]:
import re

reviews = beer_data.review

hop = []
for i in reviews:
  chopped = re.findall('[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*hop[a-z]*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*', i)
  chopped = ' '.join(chopped)
  if chopped == '':
    hop.append(None)
  else: 
    hop.append(chopped)
  
beer_data['hop'] = hop

fruit = []
for i in reviews:
  chopped = re.findall('[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*fruit[a-z]*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*', i)
  chopped = ' '.join(chopped)
  if chopped == '':
    fruit.append(None)
  else: 
    fruit.append(chopped)
  
beer_data['fruit'] = fruit

sweet = []
for i in reviews:
  chopped = re.findall('[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*sweet[a-z]*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*[\S+.!?]*\s*', i)
  chopped = ' '.join(chopped)
  if chopped == '':
    sweet.append(None)
  else: 
    sweet.append(chopped)
  
beer_data['sweet'] = sweet



In [ ]:
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
  
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 
  
    sid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
    return sentiment_dict['compound']

In [ ]:
hop_text = beer_data.hop
fruit_text = beer_data.fruit
sweet_text = beer_data.sweet

hop_sent= []
for i in hop_text:
  if i == None:
    hop_sent.append(0)
  else:
    hop_sent.append(sentiment_scores(i))

beer_data['hop_sent'] = hop_sent

fruit_sent= []
for i in fruit_text:
  if i == None:
    fruit_sent.append(0)
  else:
    fruit_sent.append(sentiment_scores(i))

beer_data['fruit_sent'] = fruit_sent

sweet_sent= []
for i in sweet_text:
  if i == None:
    sweet_sent.append(0)
  else:
    sweet_sent.append(sentiment_scores(i))

beer_data['sweet_sent'] = sweet_sent

beer_data['average_sent'] = (beer_data.hop_sent + beer_data.fruit_sent + beer_data.sweet_sent)/3
beer_data['elevation_score'] = beer_data.similarity + beer_data.average_sent



After calculating the average sentiment for each review given its sentiments surrounding the three attributes, we create a new metric, 'elvation score', to simply sum the cosine similarity and average sentiments for each review (Note that while we are doing this for each review, what we are really interested in is the average elevation score for each beer). 

## Task E
Below, we look at the average elevation score (average similarity score + average feature sentiment score) and recommend the beers with the top 3 elevation scores for the attributes hop, fruit, and sweet. The recommended beers are output below:

Double Orange Starfish - Aslin Beer Company

DFPF - J. Wakefield Brewing

Double Dry Hopped All Citra Everything - Other Half Brewing Company


In [ ]:
for i in range(3):
  display(beer_data.groupby(['product_name']).mean().sort_values(by = 'elevation_score', ascending = False).index[i])

'Double Orange Starfish - Aslin Beer Company'

'DFPF - J. Wakefield Brewing'

'Double Dry Hopped All Citra Everything - Other Half Brewing Company'

## Task F
Now we evaluate how the recommendation would change if we use word vectors to create a similarity metric as opposed to cosine similarity. We use the spacy package and the en_core_web_md word vector set to get this word vector similarity score for each beer. 



In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 2.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=241eecc0d72688636a6bb2c0ac44da06f12b98a1aa7733af7396aa1ff64e00ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-o92dwdow/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import csv
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tempfile import NamedTemporaryFile
import shutil
import warnings
import spacy
import en_core_web_md
# nlp = spacy.load('en')

In [ ]:
nlp = en_core_web_md.load()

In [ ]:

warnings.filterwarnings("ignore", category = DeprecationWarning)

attributes = 'hop fruit sweet'

reviews_df = beer_data.review
reviews_df = reviews_df.to_frame()

reviews_df['similarity'] = -1

for index, row in reviews_df.iterrows():
  reviews_df.loc[index,'similarity'] = nlp(row['review']).similarity(nlp(attributes))


We again want to make a metric based on this new similarity score as well as the sentiments for each attribute. Thus, we create another elevation score, this time with the word vector similarity instead of cosine similarity (vector_elevation = vector_similarity + average_sent).

In [ ]:
beer_data['vector_similarity'] = reviews_df['similarity']
beer_data['vector_elevation'] = beer_data.vector_similarity + beer_data.average_sent

for i in range(3):
  display(beer_data.groupby(['product_name']).mean().sort_values(by = 'vector_elevation', ascending = False).index[i])

'Emerald Grouper - Pipeworks Brewing Company'

'Rodenbach Caractère Rouge - Brouwerij Rodenbach N.V.'

'Double Dry Hopped All Citra Everything - Other Half Brewing Company'

Interestingly, the vector similarity approach still recommends the Double Dry Hopped All Citra Everything, but also recommends two different beers. 

To compare these recommendations, we can look at one more metric: what % of each beer's reviews mention an attribute specified by the customer (i.e. any of the attributes is mentioned one or more times). 

In [ ]:
def create_has(attribute1, attribute2, attribute3):
  beer_data['has ' + attribute1] = 0
  beer_data['has ' + attribute2] = 0
  beer_data['has ' + attribute3] = 0
  reviews = list(beer_data['review'].values)

  for review in reviews:
    if attribute1 in review:
      beer_data.loc[reviews.index(review), 'has ' + attribute1] = 1
    else:
      beer_data.loc[reviews.index(review), 'has ' + attribute1] = 0
    if attribute2 in review:
      beer_data.loc[reviews.index(review), 'has ' + attribute2] = 1
    else:
      beer_data.loc[reviews.index(review), 'has ' + attribute2] = 0
    if attribute3 in review:
      beer_data.loc[reviews.index(review), 'has ' + attribute3] = 1
    else:
      beer_data.loc[reviews.index(review), 'has ' + attribute3] = 0
  return beer_data

In [ ]:
beer_data = create_has('hop', 'fruit', 'sweet')

In [ ]:
def f(row):
  if row['has hop'] + row['has fruit'] + row['has sweet'] == 0:
    val = 0
  else:
    val = 1
  return val

beer_data['contains_attribute'] = beer_data.apply(f, axis = 1)

Now we can again look at the top 3 beers for cosine similarity and vector similarity, respectively and compare. 

In [ ]:
display(beer_data.groupby(['product_name']).mean().sort_values(by = 'elevation_score', ascending = False)[:3])

display(beer_data.groupby(['product_name']).mean().sort_values(by = 'vector_elevation', ascending = False)[:3])

,user_rating,hop_sent,fruit_sent,sweet_sent,average_sent,similarity,elevation_score,vector_similarity,has hop,has fruit,has sweet,contains_attribute,vector_elevation
product_name,,,,,,,,,,,,,
Double Orange Starfish - Aslin Beer Company,4.4396,0.156608,0.130364,0.228220,0.171731,0.536340,0.708071,0.682707,0.64,0.48,0.52,0.88,0.854438
DFPF - J. Wakefield Brewing,4.5008,0.000000,0.446280,0.125948,0.190743,0.474826,0.665568,0.687329,0.00,0.92,0.32,0.96,0.878072
Double Dry Hopped All Citra Everything - Other Half Brewing Company,4.4140,0.185320,0.148924,0.229356,0.187867,0.477559,0.665426,0.704441,0.56,0.60,0.56,0.92,0.892307


,user_rating,hop_sent,fruit_sent,sweet_sent,average_sent,similarity,elevation_score,vector_similarity,has hop,has fruit,has sweet,contains_attribute,vector_elevation
product_name,,,,,,,,,,,,,
Emerald Grouper - Pipeworks Brewing Company,4.2008,0.107988,0.186848,0.424560,0.239799,0.411947,0.651746,0.700011,0.52,0.56,0.68,0.88,0.939810
Rodenbach Caractère Rouge - Brouwerij Rodenbach N.V.,4.3304,0.033720,0.218068,0.364788,0.205525,0.394969,0.600494,0.700180,0.08,0.72,0.60,0.88,0.905706
Double Dry Hopped All Citra Everything - Other Half Brewing Company,4.4140,0.185320,0.148924,0.229356,0.187867,0.477559,0.665426,0.704441,0.56,0.60,0.56,0.92,0.892307


We can see that the cosine similarity approach seems to recommend 3 beers that contain the attributes at a higher rate (88%, 96% and 92%) than the word vector similarity recommendation (88%, 88%, 92%). This makes intuitive sense as the word vector similarity score will be taking into account synonyms in addition to the attributes themselves. 


## Task G
Finally, we can check the simple approach of recommending the beers with the highest ratings.

In [ ]:
display(beer_data.groupby(['product_name']).mean().sort_values(by = 'user_rating', ascending = False)[:3])

,user_rating,hop_sent,fruit_sent,sweet_sent,average_sent,similarity,elevation_score,vector_similarity,has hop,has fruit,has sweet,contains_attribute,vector_elevation
product_name,,,,,,,,,,,,,
Chemtrailmix - 3 Floyds Brewing Co.,4.754286,-0.003290,0.000000,0.180443,0.059051,0.145224,0.204275,0.659392,0.095238,0.00,0.333333,0.380952,0.718443
It Was All A Dream - J. Wakefield Brewing,4.747200,0.000000,0.018352,0.239392,0.085915,0.122186,0.208100,0.666144,0.040000,0.04,0.320000,0.360000,0.752059
SR-71 - Toppling Goliath Brewing Company,4.746400,0.017616,0.014448,0.087300,0.039788,0.157963,0.197751,0.624186,0.040000,0.12,0.240000,0.360000,0.663974


From the three beers with the highest ratings shown above, none of them were recommended by the cosine similarity or vector similarity systems. They all have relatively low occurence percentages of the attributes desired, and low elevation scores. Thus, we see the value in a recommendation system that takes attributes and sentiments into account to deliver a recommendation that matches the qualities the customer desires. 